# Load DeepFly3D Data

In [1]:
%load_ext autoreload
%autoreload 2
from load import *
import torch
import yaml
import logging
from imp import reload
import matplotlib.pyplot as plt
from liftpose.vision_3d import world_to_camera_dict, reprojection_error, intrinsic_matrix
reload(logging)
logger = logging.getLogger(__name__).setLevel(logging.INFO)
from tqdm import tqdm
tqdm.get_lock().locks = []

# decleare data parameters
par_train = {  'data_dir'       : '/data/LiftPose3D/fly_tether/data_DF3D', # change the path 
               'out_dir'        : '/data/LiftPose3D/fly_tether/angle_inv_network',
               'train_subjects' : [1],
               'test_subjects'  : [6,7],
               'actions'        : ['all'],
               'cam_id'         : [0]}

# merge with training parameters
par_data = yaml.full_load(open('param.yaml', "rb"))
par = {**par_data["data"], **par_train}

# Load 2D data
train_2d = load_2D(
    par["data_dir"],
    par,
    cam_id=par["cam_id"],
    subjects=par["train_subjects"],
    actions=par["actions"],
)
test_2d = load_2D(
    par["data_dir"],
    par,
    cam_id=par["cam_id"],
    subjects=par["test_subjects"],
    actions=par["actions"],
)

# Load 3D data
train_3d, train_keypts, rcams_train = load_3D(
    par["data_dir"],
    par,
    cam_id=par["cam_id"],
    subjects=par["train_subjects"],
    actions=par["actions"],
)
test_3d, test_keypts, rcams_test = load_3D(
    par["data_dir"],
    par,
    cam_id=par["cam_id"],
    subjects=par["test_subjects"],
    actions=par["actions"],
)

test_3d = world_to_camera_dict(test_3d, rcams_test)


In [7]:
rcams_train

{(1,
  'all',
  'pose_result_fix__data_paper_180918_MDN_CsCh_Fly1_001_SG1_behData_images.cam_0'): {'R': array([[ 0.90917256,  0.00832571, -0.41633634],
         [ 0.00513456,  0.99949997,  0.03120012],
         [ 0.41638792, -0.030504  ,  0.90867519]]),
  'tvec': array([1.64306473e+00, 3.04749148e-02, 1.17613039e+02]),
  'intr': array([[1.60410e+04, 0.00000e+00, 2.40000e+02],
         [0.00000e+00, 1.59717e+04, 4.80000e+02],
         [0.00000e+00, 0.00000e+00, 1.00000e+00]]),
  'distort': array([0., 0., 0., 0., 0.]),
  'vis': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0.])},
 (1,
  'all',
  'pose_result_fix__data_paper_180918_MDN_CsCh_Fly1_002_SG1_behData_images.cam_0'): {'R': array([[ 0.90682841,  0.01186894, -0.42133284],
         [-0.00557036,  0.99985363,  0.01617685],
         [ 0.42146317, -0.01232266,  0.90676179]]),
  'tvec': array([1.63777815e+00, 3

# Train LiftPose3D Network on DeepFly3D Data

In [4]:
from liftpose.main import train as lp3d_train
from liftpose.lifter.augmentation import random_project
from liftpose.preprocess import obtain_projected_stats,  get_visible_points

intr = intrinsic_matrix(171.0 * 94.0, 171.0 * 94.0, 240, 480)
training_kwargs = {'eangles': {0: [[-30,-20], [-2, 2], [-2,2]]},
                   'axsorder': 'yxz', 
                   'intr': intr}
#training_kwargs = {'eangles': {0: [[-90,-10], [-10, 10], [-2,2]], 1: [[-270,-180], [-10, 10], [-2,2]]},
#                   'axsorder': 'yxz', 
#                   'intr': intr}

aug = random_project(**training_kwargs)
d_train = get_visible_points(train_3d, train_keypts)
stats = obtain_projected_stats(d_train, 
                              training_kwargs['eangles'], 
                              training_kwargs['axsorder'], 
                              training_kwargs['intr'], 
                              par['roots'], 
                              par['target_sets'],
                              par['out_dir'], 
                              th=0.05)

pickle.dump([training_kwargs,stats],open('/data/LiftPose3D/fly_tether/angle_inv_network/stats.pkl','wb'))

KeyboardInterrupt: 

In [2]:
stats = pickle.load(open('/data/LiftPose3D/fly_tether/angle_inv_network/stats.pkl','rb'))

In [6]:
lp3d_train(train_2d=train_2d, test_2d=test_2d,
           train_3d=train_3d, test_3d=test_3d, 
           train_keypts=train_keypts,
           test_keypts=test_keypts,
           roots=par['roots'],
           target_sets=par['target_sets'],
           out_dir=par['out_dir'],
           augmentation=[aug],
           mean=stats)

[main.py:239]:INFO:Saving pre-processed 2D data at /data/LiftPose3D/fly_tether/angle_inv_network/stat_2d.pth.tar.
[main.py:258]:INFO:Saving pre-processed 3D data at /data/LiftPose3D/fly_tether/angle_inv_network/stat_3d.pth.tar.
[main.py:286]:INFO:Starting training model.
[main.py:296]:DEBUG:
==================Options=================
[main.py:297]:DEBUG:{   'batch_size': 64,
    'data_dir': '/data/LiftPose3D/fly_tether/angle_inv_network',
    'dropout': 0.5,
    'epochs': 30,
    'exp': '',
    'is_train': True,
    'job': 8,
    'linear_size': 1024,
    'load': None,
    'lr': 0.001,
    'lr_decay': 5000,
    'lr_gamma': 0.9,
    'max_norm': True,
    'noise': None,
    'num_stage': 2,
    'out': '/data/LiftPose3D/fly_tether/angle_inv_network',
    'out_dir': '/data/LiftPose3D/fly_tether/angle_inv_network',
    'predict': False,
    'procrustes': False,
    'resume': False,
    'test': False}
[main.py:298]:DEBUG:==========================================

[lift.py:31]:INFO:Training on

Epoch 000 | LR  0.00100 | Loss Test  0.00000 | Loss Train      nan|:  19%|█▉        | 314/1646 [00:02<00:11, 120.01it/s]


KeyboardInterrupt: 

In [ ]:
%matplotlib inline
from liftpose.plot import read_log_train, plot_log_train
epoch, lr, loss_train, loss_test, err_test = read_log_train(par['out_dir'])
plot_log_train(plt.gca(), loss_train, loss_test, epoch)

# Run Trained LiftPose3D Network on the Test Data

In [ ]:
from liftpose.main import test as lp3d_test
lp3d_test(par['out_dir'])

# Visualize Predictions

In [ ]:
from liftpose.postprocess import load_test_results
data = torch.load(os.path.join(par['out_dir'], "test_results.pth.tar"))
stat_2d, stat_3d = (
    torch.load(os.path.join(par['out_dir'], "stat_2d.pth.tar")),
    torch.load(os.path.join(par['out_dir'], "stat_3d.pth.tar")),
)
test_3d_gt, test_3d_pred, good_keypts = load_test_results(data, stat_2d, stat_3d)

In [ ]:
# https://stackoverflow.com/a/38865534/7554774
# conda install ipympl
#%matplotlib widget
%matplotlib inline
from liftpose.plot import plot_pose_3d

fig = plt.figure(figsize=plt.figaspect(1), dpi=100)
ax = fig.add_subplot(111, projection='3d')
ax.view_init(elev=90, azim=0)

t = 0
plot_pose_3d(ax=ax, tar=test_3d_gt[t], 
            pred=test_3d_pred[t], 
            bones=par_data["vis"]["bones"], 
            limb_id=par_data["vis"]["limb_id"], 
            colors=par_data["vis"]["colors"],
            good_keypts=good_keypts[t],
            show_gt_always=False)